In [1]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.metrics import classification_report
from keras.datasets import cifar10
import autokeras as ak
import os
from tqdm import tqdm

from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
from random import shuffle

In [4]:
# initialize the output directory
OUTPUT_PATH = ""
TRAINING_TIME = 60 * 60 * 1 # 1 hour

In [5]:
SIZE = 50

In [6]:
PATH = '../input/cats-vs-dogs/train/'

In [7]:
def get_label(file):
    class_label = file.split('.')[0]
    if class_label == 'dog': label_vector = 0
    elif class_label == 'cat': label_vector = 1
    return label_vector

In [8]:
def get_data():
    data = []
    files = os.listdir(PATH)
    for image in tqdm(files):
        
        label_vector = get_label(image)
        

        img = Image.open(PATH + image).convert('L')
        img = img.resize((SIZE,SIZE))
        
        data.append([np.asarray(img),np.array(label_vector)])
        
    shuffle(data)
    return data

In [9]:
data = get_data()

100%|██████████| 10000/10000 [00:42<00:00, 237.62it/s]


In [10]:
train = data[:7000]
test = data[7000:]

In [11]:
x_train = [data[0] for data in train]
y_train = [data[1] for data in train]
x_test = [data[0] for data in test]
y_test = [data[1] for data in test]

In [12]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [13]:
x_train = np.array(x_train).reshape(-1,SIZE,SIZE,1)
x_test = np.array(x_test).reshape(-1,SIZE,SIZE,1)

In [14]:
model = ak.ImageClassifier(verbose=True)
model.fit(x_train, y_train, time_limit=TRAINING_TIME)
model.final_fit(x_train, y_train, x_test, y_test, retrain=True)

Saving Directory: /tmp/autokeras_SXTBBX
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                 
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   2.409887635707855    |   0.6744000000000001   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+
Epoch-1, Current Metric - 0:  39%|███████████▊                  |

In [ ]:
model.export_autokeras_model("model.h5")

In [ ]:
from autokeras.utils import pickle_from_file
model = pickle_from_file("model.h5")

In [ ]:
predictions = model.predict(x_test)

In [15]:
score = model.evaluate(x_test, y_test)

In [17]:
print("\nScore: {}\n".format(score))


Score: 0.7186666666666667

